In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Load tokenizer and model from Hugging Face
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Load and tokenize dataset
def load_dataset(file_path, tokenizer):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=128
    )

train_dataset = load_dataset("fine_tune.txt", tokenizer)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Training arguments with wandb logging disabled
training_args = TrainingArguments(
    output_dir="./gpt2-business",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    learning_rate=5e-5,
    report_to=[]  # Disable wandb and other loggers
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator
)

# Start training
trainer.train()

# Save the fine-tuned model and tokenizer
trainer.save_model("./gpt2-business")
tokenizer.save_pretrained("./gpt2-business")


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,2.562100
200,2.055900
300,1.812000
400,1.659000
500,1.585200
600,1.470400
700,1.419300
800,1.422900


('./gpt2-business/tokenizer_config.json',
 './gpt2-business/special_tokens_map.json',
 './gpt2-business/vocab.json',
 './gpt2-business/merges.txt',
 './gpt2-business/added_tokens.json')

In [ ]:
!pip install transformers datasets


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os

model_path = "/content/gpt2-business"
try:
    tokenizer = GPT2Tokenizer.from_pretrained(model_path)
    model = GPT2LMHeadModel.from_pretrained(model_path)
    print(f"Model and tokenizer loaded successfully from {model_path}")
except Exception as e:
    print(f"Error loading model or tokenizer from {model_path}: {e}")
    print("Please ensure the model has been fine-tuned and saved correctly to this path.")
    print("If you haven't fine-tuned the model yet, please run the fine-tuning script first.")
    exit()

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("Tokenizer pad_token set to eos_token.")

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    pad_token_id=tokenizer.eos_token_id,
    truncation=True,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95
)
print("Text generation pipeline configured.")

llm = HuggingFacePipeline(pipeline=generator)

prompt_template = PromptTemplate(
    input_variables=["user_input"],
    template="User said: {user_input}\nAssistant:"
)
print("LangChain prompt template created.")

chain = LLMChain(llm=llm, prompt=prompt_template)
print("LangChain LLMChain initialized.")

print("\n--- Chatbot Ready ---")
print("Chatbot is ready! Type 'exit' or 'quit' to stop.\n")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Bot: Goodbye!")
        break

    response = chain.run(user_input)

    if isinstance(response, str):
        response = response.strip()

        full_prompt_echo = f"User said: {user_input}"

        if response.lower().startswith(full_prompt_echo.lower()):
            response = response[len(full_prompt_echo):].strip()
            response = response.lstrip(":? .\n").strip()

        for prefix in ["Assistant:", "User:", "Bot:", "user:", "bot:", "You:"]:
            while response.lower().startswith(prefix.lower()):
                response = response[len(prefix):].strip()
                response = response.lstrip(":? .\n").strip()

        lines = response.splitlines()
        cleaned_lines = []
        for line in lines:
            s = line.strip()
            if s.endswith('?') and user_input.lower() not in s.lower() and len(cleaned_lines) > 0:
                break
            if not s.lower().startswith(("user:", "bot:", "you:", "assistant:")):
                cleaned_lines.append(s)

        response = '\n'.join(cleaned_lines).strip()

        if response:
            response = response[0].upper() + response[1:]

    print("Bot:", response)

Device set to use cuda:0


Model and tokenizer loaded successfully from /content/gpt2-business
Tokenizer pad_token set to eos_token.


<ipython-input-6-7c2205bc8dbc>:36: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=generator)
<ipython-input-6-7c2205bc8dbc>:44: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)


Text generation pipeline configured.
LangChain prompt template created.
LangChain LLMChain initialized.

--- Chatbot Ready ---
Chatbot is ready! Type 'exit' or 'quit' to stop.

You: exit
Bot: Goodbye!


In [ ]:
!pip install transformers huggingface_hub git-lfs


In [ ]:
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
!pip install huggingface_hub

In [ ]:
import os
os.environ["HUGGING_FACE_HUB_TOKEN"] = "ADD TOKEN"

In [ ]:
!huggingface-cli whoami


KainatIjaz


In [ ]:
repo_name = "my-finetuned-gpt2-model"

In [ ]:
print(f"Pushing model to {repo_name}...")
model.push_to_hub(repo_name)
print("Model pushed successfully!")

Pushing model to my-finetuned-gpt2-model...


model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

Model pushed successfully!


In [ ]:
print(f"Pushing tokenizer to {repo_name}...")
tokenizer.push_to_hub(repo_name)
print("Tokenizer pushed successfully!")

Pushing tokenizer to my-finetuned-gpt2-model...


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Tokenizer pushed successfully!
